In [1]:
import io
import os
from google.cloud import vision_v1p3beta1 as vision
import cv2
import matplotlib.pyplot as plt

# ตั้งค่า Google Cloud Vision API Key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'client_key.json'

# กำหนดเส้นทางของโฟลเดอร์ที่เก็บภาพ
SOURCE_PATH = "lp.jpg"

def recognize_license_plate(img_path):
    # โหลดภาพด้วย OpenCV
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    
    # ปรับขนาดของภาพ
    img = cv2.resize(img, (800, int((height * 800) / width)))
    
    # สร้างไฟล์ชั่วคราว
    temp_img_path = os.path.join(SOURCE_PATH, "output.jpg")
    cv2.imwrite(temp_img_path, img)
    
    # สร้าง Google Vision Client
    client = vision.ImageAnnotatorClient()
    
    # อ่านไฟล์ภาพ
    with io.open(temp_img_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    
    # ตรวจจับโลโก้และข้อความในภาพ
    response = client.logo_detection(image=image)
    logos = response.logo_annotations
    response2 = client.text_detection(image=image)
    texts = response2.text_annotations
    
    # วาดผลการตรวจจับโลโก้
    for logo in logos:
        print("โลโก้ที่ตรวจพบ:", logo.description)
        cv2.putText(img, logo.description, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # วาดผลการตรวจจับป้ายทะเบียน
    if texts:
        license_plate = texts[0].description  # ป้ายทะเบียนตัวแรก
        print("ป้ายทะเบียนที่ตรวจพบ:", license_plate)
        vertices = [(vertex.x, vertex.y) for vertex in texts[0].bounding_poly.vertices]
        cv2.rectangle(img, (vertices[0][0]-10, vertices[0][1]-10), 
                      (vertices[2][0]+10, vertices[2][1]+10), (0, 255, 0), 3)
        
    # แสดงภาพที่ตรวจจับได้ใน Jupyter Notebook
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

print('---------- Start recognize license plate --------')

# ค้นหาไฟล์ภาพในโฟลเดอร์
files = [os.path.join(r, file) for r, d, f in os.walk(SOURCE_PATH) for file in f if file.endswith('.jpg')]

# รันการตรวจจับป้ายทะเบียนในภาพทั้งหมด
for file_path in files:
    print("กำลังตรวจจับป้ายทะเบียนใน:", file_path)
    recognize_license_plate(file_path)
    
print('---------- End ----------')


---------- Start recognize license plate --------
---------- End ----------
